In [168]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import Dataset
from transformers import EarlyStoppingCallback
from torch.nn import CrossEntropyLoss

In [169]:
# 检查GPU可用性
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [170]:
# 数据加载
df = pd.read_csv(r"D:\GitHubRepos\is6941-ml-social-media\taptap\data\integrated\lm_cleaned_taptap_reviews.csv")
df = df[['review_content', 'sentiment']].dropna()
df['sentiment'] = df['sentiment'].astype(int)

In [171]:
# 划分数据集
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review_content'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42
)

In [172]:
# 修正后的数据集类
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, 
                                 padding='max_length',  # 统一填充长度
                                 truncation=True, 
                                 max_length=256)
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

In [173]:
# 计算类别权重
class_weights = torch.tensor(
    [len(train_labels)/sum(train_labels),  # 正样本权重
     len(train_labels)/(len(train_labels)-sum(train_labels))],  # 负样本权重
    device=device
)

In [174]:
# 初始化模型和分词器
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
# 修改模型初始化
model = BertForSequenceClassification.from_pretrained(
    "bert-base-chinese",
    num_labels=2,
    # problem_type="single_label_classification",
    # hidden_dropout_prob=0.3,
    # classifier_dropout=0.2
)

# 将类别权重移动到GPU
class_weights = class_weights.to(device)

c:\Users\wjw20\miniconda3\envs\IS6941\Lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificatio

In [175]:
# 创建数据集（保持CPU张量）
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

In [176]:
# 训练配置（自动处理数据到GPU）
training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    fp16=True,  # 自动启用pin_memory
    dataloader_pin_memory=True,  # 显式启用内存固定
    # learning_rate=3e-5,  # 初始学习率从默认的5e-5调低
    # warmup_ratio=0.1,    # 添加学习率预热
    # weight_decay=0.01,   # L2正则化
    # gradient_accumulation_steps=2,   # 梯度累积
)

In [177]:
# 自定义评估函数
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    print("\nClassification Report:")
    print(classification_report(labels, preds))
    print("Confusion Matrix:")
    print(confusion_matrix(labels, preds))
    return {'accuracy': (preds == labels).mean()}

In [ ]:
# 创建训练器
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [179]:
# 开始训练（自动处理数据迁移）
trainer.train()

c:\Users\wjw20\miniconda3\envs\IS6941\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3000 [00:00<?, ?it/s]

{'loss': 0.441, 'learning_rate': 4.166666666666667e-05, 'epoch': 0.5}
{'loss': 0.3935, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.73      0.75      2740
           1       0.86      0.89      0.88      5257

    accuracy                           0.84      7997
   macro avg       0.82      0.81      0.82      7997
weighted avg       0.84      0.84      0.84      7997

Confusion Matrix:
[[1999  741]
 [ 560 4697]]
{'eval_loss': 0.36735033988952637, 'eval_accuracy': 0.8373139927472802, 'eval_runtime': 15.6136, 'eval_samples_per_second': 512.183, 'eval_steps_per_second': 8.006, 'epoch': 1.0}
{'loss': 0.3063, 'learning_rate': 2.5e-05, 'epoch': 1.5}
{'loss': 0.2937, 'learning_rate': 1.6683333333333333e-05, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.72      0.76      2740
           1       0.86      0.90      0.88      5257

    accuracy                           0.84      7997
   macro avg       0.83      0.81      0.82      7997
weighted avg       0.84      0.84      0.84      7997

Confusion Matrix:
[[1968  772]
 [ 501 4756]]
{'eval_loss': 0.3690436780452728, 'eval_accuracy': 0.8408153057396524, 'eval_runtime': 15.9317, 'eval_samples_per_second': 501.956, 'eval_steps_per_second': 7.846, 'epoch': 2.0}
{'loss': 0.1862, 'learning_rate': 8.350000000000001e-06, 'epoch': 2.5}
{'loss': 0.1867, 'learning_rate': 1.6666666666666667e-08, 'epoch': 3.0}


  0%|          | 0/125 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      2740
           1       0.87      0.89      0.88      5257

    accuracy                           0.84      7997
   macro avg       0.83      0.82      0.82      7997
weighted avg       0.84      0.84      0.84      7997

Confusion Matrix:
[[2072  668]
 [ 585 4672]]
{'eval_loss': 0.43909624218940735, 'eval_accuracy': 0.8433162435913467, 'eval_runtime': 15.927, 'eval_samples_per_second': 502.104, 'eval_steps_per_second': 7.848, 'epoch': 3.0}
{'train_runtime': 667.4412, 'train_samples_per_second': 143.779, 'train_steps_per_second': 4.495, 'train_loss': 0.3012294667561849, 'epoch': 3.0}


TrainOutput(global_step=3000, training_loss=0.3012294667561849, metrics={'train_runtime': 667.4412, 'train_samples_per_second': 143.779, 'train_steps_per_second': 4.495, 'train_loss': 0.3012294667561849, 'epoch': 3.0})

In [181]:
# 最终测试集评估
test_results = trainer.predict(test_dataset)
print("\nFinal Test Set Evaluation:")
compute_metrics(test_results)

  0%|          | 0/125 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      2740
           1       0.87      0.89      0.88      5257

    accuracy                           0.84      7997
   macro avg       0.83      0.82      0.82      7997
weighted avg       0.84      0.84      0.84      7997

Confusion Matrix:
[[2072  668]
 [ 585 4672]]

Final Test Set Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.76      0.77      2740
           1       0.87      0.89      0.88      5257

    accuracy                           0.84      7997
   macro avg       0.83      0.82      0.82      7997
weighted avg       0.84      0.84      0.84      7997

Confusion Matrix:
[[2072  668]
 [ 585 4672]]


{'accuracy': 0.8433162435913467}